<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Cuts Optimization using Extra Gradient Boosting
<br></p><br>

Over the last years, **Machine Learning** tools have been successfully applied to problems in high-energy physics. For example, for the classification of physics objects. Supervised machine learning algorithms allow for significant improvements in classification problems by taking into account observable correlations and by learning the optimal selection from examples, e.g. from Monte Carlo simulations.


# Importing the Libraries

**Numpy** is a powerful library that makes working with python more efficient, so we will import it and use it as np in the code. **Pandas** is another useful library that is built on numpy and has two great objects *series* and *dataframework*. Pandas works great for *data ingestion* and also has *data visualization* features. From **Hipe4ml** we import **TreeHandler** and with the help of this function we will import our *Analysis Tree* to our notebook.

**Matplotlib** comes handy in plotting data while the machine learning is performed by **XGBOOST**. We will import data splitter from **Scikit-learn** as *train_test_split*. **Evaluation metrics** such as *confusion matrix*, *Receiver operating characteristic (ROC)*, and *Area Under the Receiver Operating Characteristic Curve (ROC AUC)*  will be used to asses our models.

A **Confusion Matrix** $C$ is such that $C_{ij}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$. Thus in binary classification, the count of true positives is $C_{00}$, false negatives $C_{01}$,false positives is $C_{10}$, and true neagtives is $C_{11}$.

If $ y^{'}_{i} $ is the predicted value of the $ i$-th sample and $y_{i}$ is the corresponding true value, then the fraction of correct predictions over $ n_{samples}$ is defined as 
$$
True \: positives (y,y^{'}) =  \sum_{i=1}^{n_{samples} } 1 (y^{'}_{i} = y_{i}=1)
$$ 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization
#from root_pandas import read_root


from data_cleaning import clean_df
from KFPF_lambda_cuts import KFPF_lambda_cuts
from plot_tools import AMS, preds_prob, plot_confusion_matrix
from tree_importer import tree_importer
import uproot


#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(8)


Bad key "text.kerning_factor" on line 4 in
/home/shahid/cbmsoft/anaconda-u/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# Importing the data
CBM has a modified version of the cern's root software and it contains the simulated setup of CBM. Normally, a model generated input file, for example a URQMD 12 AGeV, is passed through different macros. These macros represent the CBM setup and it is like taking particles and passing them through a detector. These particles are registered as hits in the setup. Then particles' tracks are reconstructed from these hits using cellular automaton and Kalman Filter mathematics.


CBM uses the **tree** format of cern root to store information. To reduce the size of these root files a modified tree file was created by the name of Analysis tree. This Analysis tree file contains most of the information that we need for physics analysis. 

In this example, we download three Analysis Trees. The first one contains mostly background candidates for lambda i.e. protons and pions which do not come from a lambda. The second file contains mostly signal candidates of lamba i.e. it contains protons and pions which come from a lambda decay. The third one contains 10k events generated using URQMD generator with 12 AGeV energy.

In [2]:
# We import three root files into our jupyter notebook
signal = tree_importer('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root','PlainTree')
# We only select lambda candidates
sgnal = signal[(signal['LambdaCandidates_is_signal']==1) & (signal['LambdaCandidates_mass']>1.108)
               & (signal['LambdaCandidates_mass']<1.1227)]
del signal

In [ ]:
# Similarly for the background
background = tree_importer('/home/shahid/cbmsoft/Data/PFSimplePlainTreeBackground.root','PlainTree')
bg = background[(background['LambdaCandidates_is_signal'] == 0)
                & ((background['LambdaCandidates_mass'] > 1.07)
                & (background['LambdaCandidates_mass'] < 1.108) | (background['LambdaCandidates_mass']>1.1227) 
                   & (background['LambdaCandidates_mass'] < 2.00))]


del signal
del background

# Parallel processing

In [3]:
df_original = tree_importer('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root','PlainTree')
import gc
gc.collect()

29535

In [ ]:
df_original = tree_importer('/home/shahid/Mount/gsi/u/flat_trees/apr20_fr_18.2.1_fs_jun19p1/dcmqgsm_smm_pluto/auau/12agev/mbias/sis100_electron_target_25_mkm/PFSimplePlainTree.root','PlainTree')
import gc
gc.collect()

In [4]:
bg = df_original[(df_original['LambdaCandidates_is_signal'] == 0)
                & ((df_original['LambdaCandidates_mass'] > 1.07)
                & (df_original['LambdaCandidates_mass'] < 1.108) | (df_original['LambdaCandidates_mass']>1.1227) 
                   & (df_original['LambdaCandidates_mass'] < 2.00))]

In [ ]:
new_labels=['chi2geo', 'chi2primneg','chi2primpos', 'chi2topo','cosineneg', 'cosinepos',
       'cosinetopo', 'distance','eta', 'l', 'ldl','mass', 'masserr',
       'p', 'pT', 'phi','px', 'pxerr', 'py','pyerr', 'pz','pzerr', 'rapidity',
       'x', 'y', 'z','daughter1id','daughter2id','issignal', 'isfrompv','nhitsneg', 'nhitspos', 'pid']
df_original.columns = new_labels
bg.columns=new_labels

## Renaming the columns

In [5]:
#The labels of the columns in the df data frame are having the prefix LambdaCandidates_ so we rename them
new_labels= ['chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosineneg',
       'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl',
       'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity',
             'x', 'y', 'z', 'daughter1id', 'daughter2id', 'isfrompv', 'pid', 'issignal']

sgnal.columns = new_labels
bg.columns = new_labels

#Let's see how the dataframe object df looks like
df_original.columns=new_labels

In [ ]:
df_signal = df_original[df_original['issignal']==1]
df_bg = df_original[df_original['issignal']==0]
del df_original 

In [ ]:
df_bg['log-l']= np.log(df_bg['l'])

In [ ]:
df_signal['log-l']= np.log(df_signal['l'])

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
df_bg['log-l'].hist(bins=np.arange(-20,80,0.5),alpha=0.3)
df_signal['log-l'].hist(bins=np.arange(-20,80,0.5), alpha=0.3)
plt.yscale('log')
#plt.xscale('log')
#plt.ylabel('logarithmic scale')
plt.title('URQMD 10k events, signal candidates, l')
plt.savefig('hists')

In [ ]:
((13345-13314)/13345)*100

In [ ]:
np.exp(-4)

In [ ]:
def clean_df(df):
    import numpy as np
    import pandas as pd
    with pd.option_context('mode.use_inf_as_na', True):
        df = df.dropna()
    is_good_mom = (df['pz'] > 0) & (df['p']<20)
    is_good_coord = (abs(df['x']) < 50) & (abs(df['y']) < 50) & (df['z']>0) & (df['z']<80)
    is_good_params = (df['distance'] > 0) & (df['distance'] < 100) & (df['chi2geo']>0) & (df['chi2geo'] < 1000) & (df['cosinepos'] > 0.5) & (df['chi2topo'] > 0) & (df['chi2topo'] < 100000) & (df['cosineneg']>0.1) & (df['eta']>1) & (df['eta']<6.5) & (df['l']<80) & (df['ldl']>0) & (df['ldl']<5000)
    is_good_daughters = (df['chi2primneg']>0) & (df['chi2primneg'] < 3e7) & (df['chi2primpos']>0) & (df['chi2primpos']<1e6)
    is_good_mass = (df['mass']>1.1) & (df['mass']<2)

    is_good_df = (is_good_mom) & (is_good_coord) & (is_good_params) & (is_good_daughters) & (is_good_mass)

    return df[is_good_df]


In [ ]:
#new_signal=sgnal[((sgnal['eta']>1)) &  ((sgnal['eta']<4))]
new_signal = sgnal[(abs(sgnal['x'])<50) & (abs(sgnal['y'])<50) & (sgnal['eta']>1) & (sgnal['eta']<4)]
new_signal['eta'].hist(bins=100)
plt.savefig('hists.png')
new_signal.shape

In [ ]:
#sgnal['new_eta'].hist(bins=100)
new_signal=sgnal[((sgnal['eta']>-1*np.log(np.tan((25*np.pi)/(2*180))))) &  ((sgnal['eta']<-1*np.log(np.tan((2.5*np.pi)/(2*180)))))]

new_signal['eta'].hist(bins=100)
plt.savefig('hists.png')
#new_signal.shape

In [ ]:
df_signal = df_original[df_original['issignal']==1]
df_bg = df_original[df_original['issignal']==0]
del df_original 

In [ ]:
with pd.option_context('mode.use_inf_as_na', True):
        sgnal = df_signal.dropna()

In [ ]:
with pd.option_context('mode.use_inf_as_na', True):
        bg = df_bg.dropna()

In [ ]:
del df_signal
del df_bg

In [ ]:
sgnal['log_chi2geo'] = np.log(sgnal['chi2geo'])

In [ ]:
bg['log_chi2geo'] = np.log(bg['chi2geo'])

In [ ]:
with pd.option_context('mode.use_inf_as_na', True):
        bg = bg.dropna()

In [ ]:
fig, axs = plt.subplots(figsize = (12,10))
plt.hist(sgnal['chi2geo'], bins =300, facecolor = 'red', alpha = 0.35, range=(0,20))
#plt.hist(sgnal['log_chi2geo'], bins=300, facecolor ='blue',alpha = 0.35)

plt.yscale('log')
#plt.xscale('log')
plt.show()

The above data frame object has some columns/features and for them at the very last column the true Monte Carlos information is available. This MC information tells us whether this reconstructed particle was originally produced as a decaying particle or not. So a value of 1 means that it is a true candidate and 0 means that it is not.

# Data Cleaning
Sometimes a data set contains entries which do not make sense. For example, infinite values or NaN entries. We clean the data by removing these entries. Ofcourse, we lose some data points but these outliers sometimes cause problems when we perform analysis. 

Since our experiment is a fixed target experiment so there are certain constraints which have to be applied on the data as well.

In [6]:
#Creating a new data frame and saving the results in it after cleaning of the original dfs
#Also keeping the original one
bcknd = clean_df(bg)
signal = clean_df(sgnal)

del bg
del sgnal
gc.collect()

0

In [ ]:
df_clean = clean_df(df_original)
#del df_original
gc.collect()

In [ ]:
import matplotlib as mpl
fig, axs = plt.subplots(figsize=(15, 10))
h=plt.hist2d(signal['phi'],signal['eta'], bins=100, norm=mpl.colors.LogNorm())
cbar=fig.colorbar(h[3], ax=axs)
plt.title("Cleaned", fontsize =18)
plt.xlabel('$\phi$', fontsize=18)
plt.ylabel('$\eta$', fontsize=18)
cbar.ax.tick_params(labelsize=18)
axs.tick_params(labelsize=18)
plt.show()
fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity.png")

In [ ]:
signal['rapidity'].hist(bins=100)

In [ ]:
df_clean_signal.columns

# Selecting Background and Signal
Our sample contains a lot of background (2178718) and somewhat signal candidates (36203). For analysis we will use a signal set of 4000 candidates and a background set of 12000 candidates. The background and signal candidates will be selected by using MC information.

In [ ]:
signal1 = signal[signal['rapidity']>1.5996]
signal1

In [ ]:
# We randomly choose our signal set of 4000 candidates
signal_selected= signal.sample(n=90000)

#background = 3 times the signal is also done randomly
background_selected = bcknd.sample(n=3*(signal_selected.shape[0]))

del signal
del bcknd
gc.collect()

#Let's combine signal and background
dfs = [signal_selected, background_selected]
df_scaled = pd.concat(dfs)

# Let's shuffle the rows randomly
df_scaled = df_scaled.sample(frac=1)
del dfs, signal_selected, background_selected
# Let's take a look at the top 10 entries of the df
df_scaled.iloc[0:10,:]

y = 0.5 * (np.log(E+P/E-P))


https://cbm-wiki.gsi.de/foswiki/bin/view/PWG/CbmCollisionEnergies


y = 0.5 * (np.log((12+10)/(12-10)))


using this the rapidity is 3.1992 for Ebeam =12.04 and pbeam =12 and mid rapidity is y/2=  1.5996

In [ ]:
def pT_vs_rapidity(df, var_xaxis , var_yaxis , range_var_xaxis, range_var_yaxis):
    import matplotlib as mpl
    fig, axs = plt.subplots(figsize=(15, 10))
    h=plt.hist2d(df[var_xaxis],df[var_yaxis],range=[range_var_xaxis,range_var_yaxis], bins=10, norm=mpl.colors.LogNorm())
    cbar = fig.colorbar(h[3])
    plt.vlines(x=.8,ymin=-1,ymax=4, color='r', linestyle='-')
    plt.vlines(x=2.5,ymin=-1,ymax=4, color='r', linestyle='-')
    plt.hlines(y=0.15, xmin=-0.1, xmax=3.5, colors='b', linestyles='solid', label='')
    plt.hlines(y=1.45, xmin=-0.1, xmax=3.5, colors='b', linestyles='solid', label='')
    plt.xlabel(''+var_xaxis, fontsize=15)
    plt.ylabel(''+var_yaxis, fontsize=15)
    plt.show()
    fig.tight_layout()
    fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity.png")

In [ ]:
range1=[-0.1, 3.5]
range2=[-0.1, 3.5]

pT_vs_rapidity(signal,'rapidity','pT', range1, range2)

In [ ]:
range1 = (1.077, 1.18)
fig, axs = plt.subplots(figsize=(10, 6))
#df_scaled['mass'].plot.hist(bins = 300, range=range1,grid=True,sharey=True)
(df_scaled[df_scaled['issignal']==0])['mass'].plot.hist(bins = 300, facecolor='yellow',grid=True,range=range1)
(df_scaled[df_scaled['issignal']==1])['mass'].plot.hist(bins = 300, facecolor='magenta',grid=True, range=range1)
plt.ylabel("Counts", fontsize=15)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize= 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Test and Train Lambda Invariant Mass', fontsize = 15)
plt.legend(('Background', 'Signal'), fontsize = 15)
fig.tight_layout()
fig.savefig("signal_bac_invmass_MC.png")

# Creating Train and Test sets
To make machine learning algorithms more efficient on unseen data we divide our data into two sets. One set is for training the algorithm and the other is for testing the algorithm. If we don't do this then the algorithm can overfit and we will not capture the general trends in the data. 

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
cuts = [ 'chi2primneg', 'chi2primpos', 'ldl', 'distance', 'chi2geo']


x = df_scaled[cuts].copy()

# The MC information is saved in this y variable
y =pd.DataFrame(df_scaled['issignal'], dtype='int')

## Whole set

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
x_whole = df_clean[cuts].copy()
# The MC information is saved in this y variable
y_whole = pd.DataFrame(df_clean['issignal'], dtype='int')

# KFPF

In [ ]:
#returns a new df 
new_check_set=KFPF_lambda_cuts(df_original)
del df_original
gc.collect()

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>XGB Boost 
<br></p><br>

## Bayesian
In order to find the best parameters of XGB for our data we use Bayesian optimization. Grid search and and random search could also do the same job but bayesian is more time efficient.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=324)
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_whole, label = y_whole)
dtest1=xgb.DMatrix(x_test, label = y_test)

### Hyper parameters

*subsample* [default=1]
Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
range: (0,1]

*eta* [default=0.3, alias: learning_rate]
Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
range: [0,1]


*gamma* [default=0, alias: min_split_loss]
Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
range: [0,∞]


*alpha* [default=0, alias: reg_alpha]
L1 regularization term on weights. Increasing this value will make model more conservative.

*Lasso Regression* (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the loss function.

In [ ]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, alpha, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'alpha':alpha,
              'n_estimators': n_estimators,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'auc', 'nthread' : 7}
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    return  cv_result['test-auc-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4, 10),
                                             'gamma': (0, 1),
                                            'alpha': (2,20),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,500)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=15, init_points=8, acq='ei')
#0.9951

# XGB models

In [ ]:
max_param = xgb_bo.max['params']
param= {'alpha': max_param['alpha'], 'gamma': max_param['gamma'], 'learning_rate': max_param['learning_rate'],
        'max_depth': int(round(max_param['max_depth'],0)), 'n_estimators': int(round(max_param['n_estimators'],0))
        , 'objective': 'binary:logistic'}

#Fit/train on training data
bst = xgb.train(param, dtrain)

#predicitions on training set
bst1= bst.predict(dtrain)

#predictions on test set
bst_test = pd.DataFrame(data=bst.predict(dtest1),  columns=["xgb_preds"])
y_test=y_test.set_index(np.arange(0,bst_test.shape[0]))
bst_test['issignal']=y_test['issignal']

#ROC cures for the predictions on train and test sets
train_best, test_best = AMS(y_train, bst1,y_test, bst_test['xgb_preds'])

In [ ]:
#The first argument should be a data frame, the second a column in it, in the form 'preds'
preds_prob(bst_test,'xgb_preds', 'issignal')

In [ ]:
#Applying XGB on the 10k events data-set
df_clean['xgb_preds'] = bst.predict(dtest)
preds_prob(df_clean,'xgb_preds', 'issignal')

## Already working model

In [ ]:
ax = xgb.plot_importance(bst)
plt.rcParams['figure.figsize'] = [8, 4]
plt.show()
ax.figure.tight_layout() 
ax.figure.savefig("hits.png")

In [ ]:
def cut_visualization(cut, range1=(1.105, 1.19), bins1= 300 ):
    mask1 = df_clean['xgb_preds']>cut
    df3=df_clean[mask1]
    
    fig, ax2 = plt.subplots(figsize=(15, 10), dpi = 200)
    color = 'tab:blue'
    ax2.hist(df_clean['mass'],bins = bins1, range=range1, facecolor='blue',alpha = 0.35, label='before selection')
    ax2.set_ylabel('Counts', fontsize = 15, color=color)
    ax2.set_xlabel('Mass in GeV', fontsize = 15)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend( fontsize = 15, loc='upper left')
    
    color = 'tab:red'
    ax1 = ax2.twinx()
    ax1.hist(df3['mass'], bins = bins1, range=range1, facecolor='red',alpha = 0.35, label='Machine learning (XGB)')
    ax1.set_xlabel('Mass in GeV', fontsize = 15)
    ax1.set_ylabel('Counts ', fontsize = 15, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend( fontsize = 15,loc='upper right' )
    
    
    
    plt.title("The sample's Invariant Mass with XGB (with a cut > "+str(cut)+')', fontsize = 15)
    fig.tight_layout()
    #fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
cut_visualization(test_best)

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>train_best
df3=df_clean[mask1]
fig, axs = plt.subplots(figsize=(15, 10))

range1= (1.105, 1.14)
bins1 = 150

#xgb

df3_new = df3[df3['issignal']==1]
df3_new1 = df3[df3['issignal']==0]
df3['mass'].plot.hist(bins = bins1, range=range1, facecolor='red',alpha = 0.3,grid=True,sharey=True)
#df3_new['mass'].plot.hist(bins = 300, range=range1,facecolor='blue',alpha = 0.3,grid=True,sharey=True)
df3_new1['mass'].plot.hist(bins = bins1, range=range1,facecolor='green',alpha = 0.3,grid=True,sharey=True)
plt.legend(('XGB selected lambdas','\n False positives = \n (MC =0)\n background in \n the distribution' ), fontsize = 18, loc='upper right')
#plt.rcParams["legend.loc"] = 'upper right'
plt.title("KFPF variables + cos$\Theta_{between \ \overrightarrow{P_\Lambda} \  & \ \overrightarrow{P_{\Pi^-}}}$ + $P_T$  with a cut of %.4f "%cut3 +"on the XGB probability distribution", fontsize = 18)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 18)
plt.ylabel("Counts", fontsize = 18)
axs.tick_params(labelsize=18)
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

## Confusion Matrix

By definition a confusion matrix $C$ is such that $C_{i, j}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.

Thus in binary classification, the count of true positives is $C_{0,0}$, false positives is $C_{1,0}$, true negatives is $C_{1,1}$ and false negatives is $C_{0,1}$.

The following function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.

In [ ]:
cut1 = train_best
df_clean['xgb_preds1'] = ((df_clean['xgb_preds']>cut1)*1)
cnf_matrix = confusion_matrix(y_whole, df_clean['xgb_preds1'], labels=[1,0])
#cnf_matrix = confusion_matrix(new_check_set['issignal'], new_check_set['new_signal'], labels=[1,0])
np.set_printoptions(precision=2)
fig, axs = plt.subplots(figsize=(10, 8))
axs.yaxis.set_label_coords(-0.04,.5)
axs.xaxis.set_label_coords(0.5,-.005)
plot_confusion_matrix(cnf_matrix, classes=['signal','background'], title='Confusion Matrix for XGB for cut > '+str(cut1))
plt.savefig('confusion_matrix_extreme_gradient_boosting_whole_data.png')

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Tree visualization
<br></p><br>

In [ ]:
#fig, ax1 = plt.subplots(figsize=(15, 10), dpi = 200)
xgb.plot_tree(bst,num_trees=10)
plt.rcParams['figure.figsize'] = [20, 40]
plt.rcParams['figure.dpi']=200
plt.show()

In [ ]:
xgb.to_graphviz(xg_reg, fmap='', num_trees=0, rankdir=None, yes_color=None, no_color=None, condition_node_params=None, leaf_node_params=None)

# Cut visualization

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>cut3
df3_base=df_clean[mask1]

In [ ]:
from matplotlib import gridspec

range1= (1.0999, 1.17)


fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

ns, bins, patches=axs[0].hist((df3_base['mass']),bins = 300, range=range1,Fill=True, color='red', facecolor='red',alpha = 0.3)
ns1, bins1, patches1=axs[0].hist((new_check_set['mass']),bins = 300, Fill=True, range=range1,facecolor='blue',alpha = 0.3)
#plt.xlabel("Mass in GeV", fontsize = 15)
axs[0].set_ylabel("counts", fontsize = 15)
#axs[0].grid()
axs[0].legend(('XGBoost Selected $\Lambda$s','KFPF selected $\Lambda$s'), fontsize = 15, loc='upper right')

#plt.rcParams["legend.loc"] = 'upper right'
axs[0].set_title("The lambda's Invariant Mass histogram with KFPF and XGB selection criteria on KFPF variables", fontsize = 15)
axs[0].grid()
axs[0].tick_params(axis='both', which='major', labelsize=15)
#fig.savefig("whole_sample_invmass_with_ML.png")


hist1, bin_edges1 = np.histogram(df3_base['mass'],range=(1.09, 1.17), bins=300)
hist2, bin_edges2 = np.histogram(new_check_set['mass'],range=(1.09, 1.17), bins=300)

#makes sense to have only positive values 
diff = (hist1 - hist2)
axs[1].bar(bins[:-1],     # this is what makes it comparable
        ns / ns1, # maybe check for div-by-zero!
        width=0.001)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 15)
axs[1].set_ylabel("XGB / KFPF", fontsize = 15)
axs[1].grid()
axs[1].tick_params(axis='both', which='major', labelsize=15)

plt.show()
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

## Curve Fitting

In [ ]:
from scipy.optimize import curve_fit

# Define fit function.
def fit_function(x, amplitude, mean, stddev,c,d,e):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amplitude, mean, stddev):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))
#def signal_function( x, x0, a, gam):
#    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)

#def fit_function( x, x0, a, gam, c,d,e):
#    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)+(c+d*x+e*(x**2))

# 3.) Generate exponential and gaussian data and histograms.
data = df3['mass']
bins = np.linspace(1.108, 1.126, 70)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[600,1.115,0.001,0,0,0])
#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[1.1156,800,0.0013,107000,-100000,800000])
print(popt)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.108, 1.126, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,popt[3],popt[4],popt[5]),color='red')
plt.plot(xspace,signal_function(xspace,popt[0],popt[1],popt[2]),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
from scipy.integrate import quad
def integrand( x, x0, a, gam):
    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)

x0 = popt[0]
a = popt[1]
gam = popt[2]
I_sig = quad(integrand, popt[0]-5*popt[2], popt[0]+5*popt[2] , args=(x0, a, gam))
I_sig

In [ ]:
def integrand(x,c,d,e):
    return c+d*x+e*(x**2)


c = popt[3]
d = popt[4]
e = popt[5]
I = quad(integrand, popt[0]-5*popt[2], popt[0]+5*popt[2] , args=(c,d,e))
I_sig[0]/abs(I[0])

In [ ]:
def integrand(x, amplitude, mean, stddev,c,d,e):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))+c+d*x+e*(x**2)

amplitude = popt[0]
mean = popt[1]
stddev = popt[2]
c = popt[3]
d = popt[4]
e = popt[5]
I = quad(integrand, popt[1]-5*popt[2], popt[1]+5*popt[2] , args=(amplitude,mean,stddev,c,d,e))
I_sig[0]/(np.sqrt(I[0]))

In [ ]:
df3['mass'].hist(bins=300)

# Multi-Differential Fitting

This will be carried out in 3 steps. Step1/prefit: 

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>cut3
df3_base=df_clean[mask1]
df1 = df3_base[(df3_base['mass']<1.108)]
df2 = df3_base[df3_base['mass']>1.13]
df3 = pd.concat([df1, df2])

fig, axs = plt.subplots(figsize=(15, 10))
from scipy.optimize import curve_fit

def fit_function( x, c,d,e):
    return c+d*x+e*(x**2)

data = df3['mass']
bins = np.linspace(1.1, 1.2, 300)
data_entries_1, bins_1 = np.histogram(data, bins=bins)
bin_means = (np.histogram(data, bins, weights=data)[0] /
             np.histogram(data, bins)[0])

data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

digitized = np.digitize(data, bins)
bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]



bin_heights, bin_borders, _ = plt.hist(data, bins=bins, label='histogram')
bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
popt, pcov = curve_fit(fit_function, bin_centers, bin_heights, p0=[1., 1.115, 0.001])

x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000)

plt.plot(x_interval_for_fit, fit_function(x_interval_for_fit, *popt), label='fit')



#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[600,1.115,0.001,0,0,0])
#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[0,0,0],
#                       method='lm')
#bounds=[(-4320,6000, -4000),
#                               (-3000,7570, -3100) ]

c0, d0, e0 = popt
print(popt)
dc0, dd0, de0 = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = bin_heights - fit_function(bin_centers, c0, d0, e0)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)

# 6.)
# Generate enough x values to make the curves look smooth.
#xspace = np.linspace(1.08, 1.2, 1000000)

# Plot the histogram and the fitted function.
#plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
#plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


#plt.plot(xspace,background_function(xspace,popt[0],popt[1],popt[2]),color='red')

# Make the plot nicer.
#plt.xlim(1.108,1.2)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
axs.text(0.7, 0.95, 'c = {0:0.1f}$\pm${1:0.1f}'
         .format(c0, dc0), transform = axs.transAxes)
axs.text(0.7, 0.90, 'd = {0:0.1f}$\pm${1:0.1f}'
         .format(d0, dd0), transform = axs.transAxes)
axs.text(0.7, 0.85, 'e = {0:0.1e}$\pm${1:0.1e}'
         .format(d0, de0), transform = axs.transAxes)
axs.text(0.7, 0.80, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs.transAxes)
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
from lmfit import Model
x=binscenters, 
y=data_entries

#def gaussian(x, amp, cen, wid, c,d,e):
#    """1-d gaussian: gaussian(x, amp, cen, wid)"""
#    return ((amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2)))+c+d*x+e*(x**2)


gmodel = Model(fit_function)
#result = gmodel.fit(y, x=x, amp=3, mean=1.115, std=0.0009, c=20, d=10, e=-10,  method='bfgs')
result = gmodel.fit(y, x=x, c=c0, d=d0, e=e0, method='cg')

print(result.fit_report())

fig, axs = plt.subplots(figsize=(15, 10))
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
#plt.plot(x, result.init_fit, 'k--', label='initial fit')
#plt.plot(x, result.best_fit, 'r-', label='best fit')
plt.plot(x_interval_for_fit, fit_function(x_interval_for_fit, result.values['c'],result.values['d'],result.values['e']))


## Step 2
Fixing some signal parameters

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>cut3
df3_base=df_clean[mask1]
std_of_gaus = 0.0014
# Define fit function.
def fit_function(x, amp,c,d,e):
    return (amp/(np.sqrt(2*np.pi) * std_of_gaus)) * (np.exp(-(0.5)*((x - 1.1157) / std_of_gaus)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amp):
    return (amp/(np.sqrt(2*np.pi) * std_of_gaus)) * (np.exp(-(0.5)*((x - 1.1157) / std_of_gaus)**2))
#def signal_function( x, a):
#    return a * (0.0016/2) / ((0.5)* 0.0016**2 + ( x - 1.115683 )**2)

#def fit_function( x, a, c,d,e):
#    return a * (0.0016/2) / ((0.5)* (0.0016**2) + ( x - 1.115683 )**2)+(c+d*x)++e*(x**2)

# 3.) Generate exponential and gaussian data and histograms.
data = df3_base['mass']
bins = np.linspace(1.1, 1.13, 300)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[0,c0,d0,e0])

amp1, c1, d1, e1 = popt
print(popt)
damp1, dc1, dd1, de1 = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = data_entries - fit_function(binscenters, amp1, c1, d1, e1)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)


#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[1,-5578.7,10271.68,-4489.33],
#                      bounds=[(-1000,-6000,8000,-5000),(1000,-4000,11000,-4050)])

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.10, 1.13, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,popt[1],popt[2],popt[3]),color='red')
plt.plot(xspace,signal_function(xspace,popt[0]),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
axs.text(0.7, 0.95, 'amp = {0:0.1f}$\pm${1:0.1f}'
         .format(amp1, damp1), transform = axs.transAxes)
axs.text(0.7, 0.90, 'c = {0:0.1f}$\pm${1:0.1f}'
         .format(c1, dc1), transform = axs.transAxes)
axs.text(0.7, 0.85, 'd = {0:0.1f}$\pm${1:0.1f}'
         .format(d1, dd1), transform = axs.transAxes)
axs.text(0.7, 0.80, 'e = {0:0.1e}$\pm${1:0.1e}'
         .format(d1, de1), transform = axs.transAxes)
axs.text(0.7, 0.75, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs.transAxes)
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
from lmfit import Model
x=binscenters, 
y=data_entries

gmodel = Model(fit_function)

result = gmodel.fit(y, x=x, amp=amp1, c=c1, d=c1, e=e1,  method='lsq')
#result = gmodel.fit(y, x=x, a=900, mean=1.1157, gam=0.0009, c=-100, d=100, e=10, method='lsq')

print(result.fit_report())

fig, axs = plt.subplots(figsize=(15, 10))
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
#plt.plot(x, result.init_fit, 'k--', label='initial fit')
#plt.plot(x, result.best_fit, 'r-', label='best fit')
plt.plot(xspace, fit_function(xspace, result.values['amp'],result.values['c'],result.values['d'],
                             result.values['e']))

plt.plot(xspace,background_function(xspace,result.values['c'],result.values['d'],result.values['e']),color='red')
plt.plot(xspace,signal_function(xspace,result.values['amp'])
         ,color='green')
#plt.legend(loc='best')
#plt.show()
# <end examp

## Step 3

In [ ]:
def fit_function(x, amp, mean,c,d,e):
    return (amp/(np.sqrt(2*np.pi) * std_of_gaus)) * (np.exp(-(0.5)*((x - mean) / std_of_gaus)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amp, mean):
    return (amp/(np.sqrt(2*np.pi) * std_of_gaus)) * (np.exp(-(0.5)*((x - mean) / std_of_gaus)**2))
#def signal_function( x, a,mean, gam):
#    return a * (gam/2) / ((0.5)* gam**2 + ( x - mean )**2)

#def fit_function( x, a, mean, gam, c,d,e):
#    return a * (gam/2) / ((0.5)* (gam**2) + ( x - mean )**2)+(c+d*x)++e*(x**2)

# 3.) Generate exponential and gaussian data and histograms.
data = df3_base['mass']
bins = np.linspace(1.1, 1.13, 300)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
                       p0=[amp1,1.11568,c1,d1,e1])

#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
#                       p0=[1.61,1.1156,0.0018,-4.81e+03,8.82e+03,-4.05e+03])
amp2, mean2, c2, d2, e2 = popt
print(popt)
damp2,dmean2, dc2, dd2, de2 = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = data_entries - fit_function(binscenters, amp2,mean2, c2, d2, e2)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.1, 1.13, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,c2,d2,e2),color='red')
plt.plot(xspace,signal_function(xspace,amp2,mean2),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
axs.text(0.7, 0.95, 'amp = {0:0.1f}$\pm${1:0.1f}'
         .format(amp2, damp2), transform = axs.transAxes)
axs.text(0.7, 0.85, 'mean = {0:e}$\pm${1:0.1f}'
         .format(mean2, dmean2), transform = axs.transAxes)
axs.text(0.7, 0.80, 'c = {0:0.1f}$\pm${1:0.1f}'
         .format(c2, dc2), transform = axs.transAxes)
axs.text(0.7, 0.75, 'd = {0:0.1f}$\pm${1:0.1f}'
         .format(d2, dd2), transform = axs.transAxes)
axs.text(0.7, 0.70, 'e = {0:0.1e}$\pm${1:0.1e}'
         .format(e2, de2), transform = axs.transAxes)
axs.text(0.7, 0.65, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs.transAxes)
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
def fit_function(x, amp, std,c,d,e):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - 1.115693) / std)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amp, std):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - 1.115693) / std)**2))
#def signal_function( x, a,mean, gam):
#    return a * (gam/2) / ((0.5)* gam**2 + ( x - mean )**2)

#def fit_function( x, a, mean, gam, c,d,e):
#    return a * (gam/2) / ((0.5)* (gam**2) + ( x - mean )**2)+(c+d*x)++e*(x**2)

# 3.) Generate exponential and gaussian data and histograms.
data = df3_base['mass']
bins = np.linspace(1.1, 1.13, 300)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
                       p0=[amp1,0.0015,c1,d1,e1])

#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
#                       p0=[1.61,1.1156,0.0018,-4.81e+03,8.82e+03,-4.05e+03])
amp2, std2, c2, d2, e2 = popt
print(popt)
damp2,dstd2, dc2, dd2, de2 = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = data_entries - fit_function(binscenters, amp2,std2, c2, d2, e2)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.1, 1.13, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,c2,d2,e2),color='red')
plt.plot(xspace,signal_function(xspace,amp2,std2),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
axs.text(0.7, 0.95, 'amp = {0:0.1f}$\pm${1:0.1f}'
         .format(amp2, damp2), transform = axs.transAxes)
axs.text(0.7, 0.90, 'std = {0:e}$\pm${1:0.1f}'
         .format(std2, dstd2), transform = axs.transAxes)
axs.text(0.7, 0.85, 'c = {0:0.1f}$\pm${1:0.1f}'
         .format(c2, dc2), transform = axs.transAxes)
axs.text(0.7, 0.80, 'd = {0:0.1f}$\pm${1:0.1f}'
         .format(d2, dd2), transform = axs.transAxes)
axs.text(0.7, 0.75, 'e = {0:0.1e}$\pm${1:0.1e}'
         .format(e2, de2), transform = axs.transAxes)
axs.text(0.7, 0.70, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs.transAxes)
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
from matplotlib import gridspec
fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

# Define fit function.
def fit_function(x, amp, mean, std,c,d,e):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amp, mean, std):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))
#def signal_function( x, amp,mean, gam):
#    return amp * (gam/2) / ((0.5)* gam**2 + ( x - mean )**2)

#def fit_function( x, amp, mean, gam, c,d,e):
#    return amp * (gam/2) / ((0.5)* (gam**2) + ( x - mean )**2)+(c+d*x)++e*(x**2)


data = df3_base['mass']
bins = np.linspace(1.08, 1.2, 300)
data_entries_1, bins_1 = np.histogram(data, bins=bins)



data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
                       p0=[par2[0],par2[1],par2[2],par2[3],par2[4],par2[5]], method='dogbox')

#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, 
#                       p0=[1.61,1.1156,0.0018,-4.81e+03,8.82e+03,-4.05e+03])
amp2, mean2, std2, c2, d2, e2 = popt
print(popt)
damp2, dmean2,dstd2, dc2, dd2, de2 = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = data_entries - fit_function(binscenters, amp2,mean2,std2, c2, d2, e2)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.08, 1.2, 1000000)

# Plot the histogram and the fitted function.
#axs[0].plot(binscenters, data_entries, color='navy',alpha = 0.5, label=r'Histogram entries')
axs[0].errorbar(x=binscenters, y=data_entries, yerr=(np.sqrt(data_entries)), linestyle='none', marker='.',mfc='red', ms=10, label='Bin data with $\sqrt{bin\ count}$')
#axs[0].plot(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
axs[0].plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, 
         label='Fitted function = $\dfrac{A}{\sigma\sqrt{2\pi}}\ e^{\dfrac{1}{2}\ \dfrac{(x-\mu)^2}{\sigma^2}} +c+dx+ex^2$')

#(amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))+c+d*x+e*(x**2)


axs[0].plot(xspace,background_function(xspace,c2,d2,e2),color='red',label='Background = $c+dx+ex^2$')
axs[0].plot(xspace,signal_function(xspace,amp2,mean2,std2),color='green',label='Signal= $\dfrac{A}{\sigma\sqrt{2\pi}}\ e^{\dfrac{1}{2}\ \dfrac{(x-\mu)^2}{\sigma^2}}$')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
axs[0].set_title(r'Lamda baryon')
axs[0].legend(loc='upper left')
axs[0].text(0.7, 0.95, 'amp = {0:0.1f}$\pm${1:0.1f}'
         .format(amp2, damp2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.90, 'mean = {0:e}$\pm${1:0.1f}'
         .format(mean2, dmean2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.85, 'std = {0:e}$\pm${1:0.1f}'
         .format(std2, dstd2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.80, 'c = {0:0.1f}$\pm${1:0.1f}'
         .format(c2, dc2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.75, 'd = {0:0.1f}$\pm${1:0.1f}'
         .format(d2, dd2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.70, 'e = {0:0.1e}$\pm${1:0.1e}'
         .format(e2, de2), transform = axs[0].transAxes)
axs[0].text(0.7, 0.65, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs[0].transAxes)

axs[1].plot(binscenters,(data_entries - fit_function(binscenters,*popt))/np.sqrt(data_entries))
axs[1].hlines(y = 0,xmin=1,xmax=1.3, color='r', linestyle='-')
axs[1].grid()
axs[1].set_ylabel('$\dfrac{y-f(x)}{\sqrt{bin\ count}}$')

axs[0].set_xlim([1.107,1.125])
axs[1].set_xlim([1.107,1.125])
plt.show()
plt.clf()
fig.tight_layout()
fig.savefig('hists.png')
#pcov

In [ ]:
data_entries_1

In [ ]:
mids = 0.5*(bins[1:] + bins[:-1])
mids

In [ ]:
from lmfit import Model
x=binscenters, 
y=data_entries


from matplotlib import gridspec
fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

#def gaussian(x, amp, cen, wid, c,d,e):
#    """1-d gaussian: gaussian(x, amp, cen, wid)"""
#    return ((amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2)))+c+d*x+e*(x**2)

gmodel = Model(fit_function)

result = gmodel.fit(y, x=x, amp=par2[0], mean=par2[1], std=par2[2], c=par2[3], d=par2[4], e=par2[5], method='powel')
#result = gmodel.fit(y, x=x, a=900, mean=1.1157, gam=0.0009, c=-100, d=100, e=10, method='lsq')

print(result.fit_report())


axs[0].errorbar(x=binscenters, y=data_entries, yerr=(np.sqrt(data_entries)), linestyle='none', marker='.',mfc='red', ms=10, label='Bin data with $\sqrt{bin\ count}$')
#plt.plot(x, result.init_fit, 'k--', label='initial fit')
#plt.plot(x, result.best_fit, 'r-', label='best fit')

axs[0].plot(xspace, fit_function(xspace, result.values['amp'],result.values['mean'],result.values['std']
                          ,result.values['c'],result.values['d'],result.values['e']),color='darkorange', linewidth=2.5, 
         label='Fitted function = $\dfrac{A}{\sigma\sqrt{2\pi}}\ e^{\dfrac{1}{2}\ \dfrac{(x-\mu)^2}{\sigma^2}} +c+dx+ex^2$')

axs[0].plot(xspace,background_function(xspace,result.values['c'],result.values['d'],result.values['e']),color='red',label='Background = $c+dx+ex^2$')
axs[0].plot(xspace,signal_function(xspace,result.values['amp'],result.values['mean'],result.values['std'])
         ,color='green',label='Signal= $\dfrac{A}{\sigma\sqrt{2\pi}}\ e^{\dfrac{1}{2}\ \dfrac{(x-\mu)^2}{\sigma^2}}$')
#axs[0].plot(binscenters, result.init_fit[0], 'k--', label='initial fit')


fitted_func = fit_function(binscenters, result.values['amp'],result.values['mean'],result.values['std']
                          ,result.values['c'],result.values['d'],result.values['e'])
axs[1].plot(binscenters,(data_entries - fitted_func)/np.sqrt(data_entries))
axs[1].hlines(y = 0,xmin=1,xmax=1.3, color='r', linestyle='-')
axs[1].grid()
axs[1].set_ylabel('$\dfrac{y-f(x)}{\sqrt{bin\ count}}$')

axs[0].legend(loc='upper left')
axs[0].set_xlim([1.107,1.125])
axs[1].set_xlim([1.107,1.125])
axs[1].set_ylim([-4,7])
plt.show()
#plt.show()
# <end examp

In [ ]:
#def signal_function( x, a,mean, gam):
#    return a * (gam/2) / ((0.5)* (gam**2) + ( x - mean)**2)

def signal_function( x, amp, mean, std):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))

# 3.) Generate exponential and gaussian data and histograms.
data = df_scaled[df_scaled['issignal']==1]['mass']
bins = np.linspace(1.105, 1.126, 300)

bins0 = np.linspace(1.105, 1.112, 10).reshape((10,1))
bins1 = np.linspace(1.112, 1.12, 60).reshape((60,1))
bins2 = np.linspace(1.12, 1.126,10).reshape((10,1))
bins3 = np.concatenate((bins0,bins1,bins2)).reshape((80,1))
#bins =bins3.flatten()
#bins3 = np.concatenate(bins0,bins1,bins2)
#bins = np.array(bins3).reshape(80,1)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

#popt, pcov = curve_fit(signal_function, xdata=binscenters, ydata=data_entries, p0=[10,1.115,0.0001],
#                      method='trf')

popt, pcov = curve_fit(signal_function, xdata=binscenters, ydata=data_entries, p0=[0,1.1156,0.1],
                      method='trf')

amp, mean, std = popt
print(popt)
damp, dmean, dstd = \
          [np.sqrt(pcov[j,j]) for j in range(popt.size)]

resids = data_entries - signal_function(binscenters, amp, mean, std)
redchisqr = ((resids)**2).sum()/float(bins_1.size-3)
print(redchisqr)
# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.105, 1.126, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')

plt.plot(xspace,signal_function(xspace,amp,mean,std),color='green', label='Fit')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
axs.text(0.7, 0.95, 'amp = {0:0.1f}$\pm${1:0.1f}'
         .format(amp, damp), transform = axs.transAxes)
axs.text(0.7, 0.90, 'mean = {0:e}$\pm${1:0.1f}'
         .format(mean, dmean), transform = axs.transAxes)
axs.text(0.7, 0.85, 'std = {0:e}$\pm${1:0.1e}'
         .format(std, dstd), transform = axs.transAxes)
axs.text(0.7, 0.80, ' $\chi^{2}_{reduced} $ = '+str(redchisqr), transform = axs.transAxes)
plt.show()
plt.clf()
fig.savefig('hists.png')

# PyRoot

In [ ]:
import sys, ROOT

In [ ]:
import ROOT
from ROOT import TF1, TCanvas,TMath, TColor
%jsroot on
h = ROOT.TH1F("gauss","Example histogram",300,1.08,1.2)
for i in range(0,df3_base['mass'].shape[0]):
    h.Fill(df3_base['mass'].iloc[i])

f = TF1("total","[0]*exp(-0.5*((x-[1])/[2])^2)+[3]+[4]*x+[5]*x*x",1.08,1.2);
f.SetNpx(1000);
f.SetParameters(13852.2,1.11806,0.00446980,-0.0271897,-0.0266959,-0.0260058);
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
h.Fit(f,"RNI");
print("chi2",f.GetChisquare())
par = f.GetParameters()

c.Draw()


h.Draw()
f.Draw("SAME")

## Step 1

In [ ]:
df3_base=df_clean[df_clean['xgb_preds']>test_best]
low_pt_below_mid_rapidity_cut = df3_base[(df3_base['rapidity']<1.5996) & (df3_base['pT']<0.52)]
df1 = low_pt_below_mid_rapidity_cut[(low_pt_below_mid_rapidity_cut['mass']<1.108)]
df2 = low_pt_below_mid_rapidity_cut[low_pt_below_mid_rapidity_cut['mass']>1.13]
df3 = pd.concat([df1, df2])

data = df3['mass']
h = ROOT.TH1F("Background","Background without peak",300,1.08,1.2)
for i in range(0,data.shape[0]):
    h.Fill(data.iloc[i])

f = TF1("total","[0]+[1]*x+[2]*x*x",1.08,1.2);
f.SetNpx(1000);
f.SetParameters(13852.2,1.11806,0.00446980,-0.0271897,-0.0266959,-0.0260058);
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
h.Fit(f,"RNIFCWW");
print("chi2",f.GetChisquare()/f.GetNDF())
par = f.GetParameters()

c.Draw()


h.Draw()
f.Draw("SAME")

## Step 2

In [ ]:
data = low_pt_below_mid_rapidity_cut['mass']
h = ROOT.TH1F("B_and_S","Background with signal some free parameters",300,1.08,1.2)
for i in range(0,data.shape[0]):
    h.Fill(data.iloc[i])


f1 = TF1("total","[0]*exp(-0.5*((x-1.115683)/0.0014)^2)+[1]+[2]*x+[3]*x*x",1.08,1.2);
f.SetNpx(1000);
f1.SetParameters(1,par[0],par[1],par[2]);
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
h.Fit(f1,"RNI");
print("chi2",f.GetChisquare()/f2.GetNDF())
par1 = f1.GetParameters()

c.Draw()


h.Draw("E1")
f1.Draw("SAME")

In [ ]:
%jsroot on
data = low_pt_below_mid_rapidity_cut['mass']
#ROOT.gStyle.SetOptStat(0);
h1 = ROOT.TH1F("B_&_S","rapidity<1.5996 & pT<0.4",100,1.10,1.2)
h2 = ROOT.TH1F("h2", "", 100, 1.10, 1.2);
h3 = ROOT.TH1F("h2", "", 100, 1.10, 1.2);
for i in range(0,data.shape[0]):
    h1.Fill(data.iloc[i])

pi=TMath.Pi()
f2 = TF1("full","[0]*exp(-0.5*((x-[1])/[2])^2)+[3]+[4]*x+[5]*x*x",1.08,1.2);
f2.SetNpx(1000);
f2.SetParameters(par1[0],1.116,0.001,par1[1],par1[2],par1[3]);
f2.SetLineColor(ROOT.kRed)
c = ROOT.TCanvas("myCanvasName","The Canvas Title",1000,1000)
h1.Fit(f2,"MNIR");
#TVirtualFitter::SetDefaultFitter("Minuit");
#h1.SetFillColor(kGreen);
h1.SetFillStyle(3003);
h1.SetLineWidth(2)
h1.SetStats (0)
h1.SetYTitle("Entries")
h1.SetLabelSize(0.04)
h1.SetLineColor(ROOT.kBlack)
h1.GetXaxis().SetTitle("Mass #frac{[GeV]}{c^2}")
#h2.SetMarkerColor(2)
#h1.SetTitle("Entries with Gaussian plus 2nd order polynom bac")
c.Clear()
hs.Add(h1);
c.Divide (1,2)
c.Draw()
c.cd(1);
h1.Draw("pe")
fs = TF1("fs","[0]*exp(-0.5*((x-[1])/[2])^2)",1.08,1.2);
fs.SetNpx(1000);
fs.SetLineColor(ROOT.kGreen)
fb = TF1("fb","[0]+[1]*x+[2]*x*x",1.08,1.2)
fb.SetLineStyle(2)
fb.SetLineColor(ROOT.kBlue)
fb.SetNpx(1000);
fs.SetParameters(f2.GetParameter(0),f2.GetParameter(1),f2.GetParameter(2));
fb.SetParameters(f2.GetParameter(3),f2.GetParameter(4),f2.GetParameter(5));
fs.Draw("SAME")
fb.Draw("SAME")
f2.Draw("SAME")
#h1.Sumw2();

bin1 = h1.FindBin(1.1);
bin2 = h1.FindBin(1.2);
print("red chi2",f2.GetChisquare()/((bin2-bin1+1)-6))
par2 = f2.GetParameters()
for i in range(bin1,bin2):
    f_value= f2.Eval(h1.GetBinCenter(i));
    t_value = h1.GetBinContent(i)
    h2.SetBinContent(i,f_value)
    h3.SetBinContent(i,(t_value-f_value)/h1.GetBinError(i))

#h1.Draw("SAME")
#h1.SetLineColor(kRed);
h2.Sumw2()

hs.Add(h2);
#h2.Draw("SAME")
#hs.Draw("nostack")
#h2.Draw("SAME")

integral_min = f2.GetParameter(1) - (TMath.Abs(3*f2.GetParameter(2)));
integral_max = f2.GetParameter(1) + (TMath.Abs( 3*f2.GetParameter(2)));



#To integrate over the gaussian peak we take the integral limits 3 sigmas (i.e. parameter 3) below the mean value
#(i.e. par 1) of the gaussian as a minimum limit and 3 sigmas above the mean as a max limit of the integral*/

integral_min = f2.GetParameter(1) - (TMath.Abs(3*f2.GetParameter(2)));
integral_max = f2.GetParameter(1) + (TMath.Abs( 3*f2.GetParameter(2)));


#To integrate area under the signal plus background curve we take 3 sigma and integrate
binwidth = h1.GetXaxis().GetBinWidth(1);
tot = f2.Integral(integral_min,integral_max)/binwidth;
print(tot)

#To find the signal, we integrate just the gaussian peak with 3 sigma 
signal_under_peak = fs.Integral(integral_min,integral_max)/binwidth;
print(signal_under_peak)

#Background
backgnd_under_peak = tot-signal_under_peak;
print(backgnd_under_peak)

#Significance = signal/(signal+background)^0.5
Significance = signal_under_peak/TMath.Sqrt(tot);
print(Significance)

legend = ROOT.TLegend(0.9,0.9,0.6,0.5);
legend.AddEntry(h1,"Invariant mass of lambda","l");
legend.AddEntry(f2,"Ae^{-#frac{1}{2} #frac{(x-#mu)^{2}}{#sigma^{2}}}+b+cx+dx^{2}","l");
legend.AddEntry(fs,"Ae^{-#frac{1}{2} #frac{(x-#mu)^{2}}{#sigma^{2}}}","l");
legend.AddEntry(fb,"b+cx+dx^{2}","l");
#legend.AddEntry((ROOT.TObject), ROOT.Form("Total : %f"%tot));
#legend.AddEntry((TObject), ROOT.Form("Signal : %f"%signal_under_peak));
#legend.AddEntry((TObject), ROOT.Form("Background : %f "%backgnd_under_peak));
#legend.AddEntry((TObject), ROOT.Form("Significance in sigma: %f"%Significance));

legend.Draw()
#rp = ROOT.TRatioPlot(h1, h2);
#rp.Draw("E1");
#h1.Sumw2();
c.cd(2)
#h3=(h1-h2)
h3.SetLineColor(TColor.GetColor(5))
h3.SetYTitle("D-f/dl")
h3.Draw()
f0 = TF1("f0","[0]",1.08,1.2)
f0.SetParameters(0)
f0.Draw("SAME")







c.Update()
#c.SaveAs("picture.png")

In [ ]:
%jsroot on
data = low_pt_below_mid_rapidity_cut['mass']

canvas = ROOT . TCanvas (" canvas ","", 1200,1000)
canvas .Clear ()
canvas.Draw()
pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.3 ,1 ,1)
pad1 . Draw ()
pad1 . cd ()
pad1. Clear()


#ROOT.gStyle.SetOptStat(0);
h1 = ROOT.TH1F("B_&_S","rapidity<1.5996 & pT<0.5",100,1.10,1.2)
h1.SetTitleOffset(-1)
h1.SetFillStyle(3003);
h1.SetLineWidth(2)
h1.SetStats (0)
h1.SetYTitle("Entries")
#h1.SetLabelSize(0.0)
h1.SetLineColor(ROOT.kBlack)
#h1.GetXaxis().SetTitle("Mass #frac{[GeV]}{c^2}")
h2 = ROOT.TH1F("h2", "", 100, 1.10, 1.2);
h3 = ROOT.TH1F("h2", "", 100, 1.10, 1.2);
h3.SetLineWidth(2)
h3.SetStats (0)
#h3.SetLabelSize(0.11)
h3.GetXaxis().SetTitle("Mass (GeV/c^2)")
for i in range(0,data.shape[0]):
    h1.Fill(data.iloc[i])

pi=TMath.Pi()
f2 = TF1("full","[0]*exp(-0.5*((x-[1])/[2])^2)+[3]+[4]*x+[5]*x*x",1.08,1.2);
f2.SetNpx(100000);
f2.SetParameters(par1[0],1.115,0.001,par1[1],par1[2],par1[3]);
f2.SetLineColor(ROOT.kRed)
h1.Fit(f2,"MNIR");


h1.Draw("pe")
fs = TF1("fs","[0]*exp(-0.5*((x-[1])/[2])^2)",1.08,1.2);
fs.SetNpx(100000);
fs.SetLineColor(ROOT.kGreen)
fb = TF1("fb","[0]+[1]*x+[2]*x*x",1.08,1.2)
fb.SetLineStyle(4)
fb.SetLineColor(ROOT.kBlue)
fb.SetNpx(100000);
fs.SetParameters(f2.GetParameter(0),f2.GetParameter(1),f2.GetParameter(2));
fb.SetParameters(f2.GetParameter(3),f2.GetParameter(4),f2.GetParameter(5));
fs.Draw("SAME")
fb.Draw("SAME")
f2.Draw("SAME")

#h1.Sumw2();

bin1 = h1.FindBin(1.1);
bin2 = h1.FindBin(1.2);

print("red chi2",f2.GetChisquare()/((bin2-bin1+1)-6))
chi2 = f2 . GetChisquare ()
ndf = f2 . GetNDF ()
par2 = f2.GetParameters()
for i in range(bin1,bin2):
    f_value= f2.Eval(h1.GetBinCenter(i));
    t_value = h1.GetBinContent(i)
    h2.SetBinContent(i,f_value)
    h3.SetBinContent(i,(t_value-f_value)/h1.GetBinError(i))

#h1.Draw("SAME")
#h1.SetLineColor(kRed);
h2.Sumw2()

hs.Add(h2);
#h2.Draw("SAME")
#hs.Draw("nostack")
#h2.Draw("SAME")

integral_min = f2.GetParameter(1) - (TMath.Abs(3*f2.GetParameter(2)));
integral_max = f2.GetParameter(1) + (TMath.Abs( 3*f2.GetParameter(2)));



#To integrate over the gaussian peak we take the integral limits 3 sigmas (i.e. parameter 3) below the mean value
#(i.e. par 1) of the gaussian as a minimum limit and 3 sigmas above the mean as a max limit of the integral*/

integral_min = f2.GetParameter(1) - (TMath.Abs(3*f2.GetParameter(2)));
integral_max = f2.GetParameter(1) + (TMath.Abs( 3*f2.GetParameter(2)));


#To integrate area under the signal plus background curve we take 3 sigma and integrate
binwidth = h1.GetXaxis().GetBinWidth(1);
tot = f2.Integral(integral_min,integral_max)/binwidth;
print(tot)

#To find the signal, we integrate just the gaussian peak with 3 sigma 
signal_under_peak = fs.Integral(integral_min,integral_max)/binwidth;
print(signal_under_peak)

#Background
backgnd_under_peak = tot-signal_under_peak;
print(backgnd_under_peak)

#Significance = signal/(signal+background)^0.5
Significance = signal_under_peak/TMath.Sqrt(tot);
print(Significance)

std = f2 . GetParameter (2)
latex = ROOT . TLatex ()
latex . SetNDC ()
latex . SetTextSize (0.02)
latex . DrawText (0.4 ,0.45, " Significance = %.1f /sqrt(%.1f+%.1f)= %.1f"%(signal_under_peak,signal_under_peak,backgnd_under_peak,Significance ))
latex . DrawText (0.4 ,0.35, " Std = %.4f GeV"%( std ))
latex . DrawText (0.4 ,0.25," chi2 / ndof = %.1f/%d = %.1f"%(
chi2 , ndf , chi2 / ndf ))

legend = ROOT.TLegend(0.9,0.8,0.6,0.5);
legend.AddEntry(h1,"Invariant mass of lambda","lep");
legend.AddEntry(f2,"Ae^{#frac{-1}{2} #frac{(x-#mu)^{2}}{#sigma^{2}}}+b+cx+dx^{2}","l");
legend.AddEntry(fs,"Ae^{#frac{-1}{2} #frac{(x-#mu)^{2}}{#sigma^{2}}}","l");
legend.AddEntry(fb,"b+cx+dx^{2}","l");
legend . SetLineWidth (0)
#legend.AddEntry((ROOT.TObject), ROOT.Form("Total : %f"%tot));
#legend.AddEntry((TObject), ROOT.Form("Signal : %f"%signal_under_peak));
#legend.AddEntry((TObject), ROOT.Form("Background : %f "%backgnd_under_peak));
#legend.AddEntry((TObject), ROOT.Form("Significance in sigma: %f"%Significance));

legend.Draw()
#rp = ROOT.TRatioPlot(h1, h2);
#rp.Draw("E1");
#h1.Sumw2();


canvas . cd ()
pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.3)
pad2 . Draw ()
pad2 . cd ()
pad2.Clear()

#h3=(h1-h2)
h3.SetLineColor(TColor.GetColor(5))
h3.SetYTitle("d-f/#Deltad")
h3.Draw()
line = ROOT . TLine (1.1,0 ,1.2 ,0)
line . SetLineColor ( ROOT . kRed )
line . SetLineWidth (2)
line . Draw (" same ")


pad1 . SetBottomMargin (0)
pad2 . SetTopMargin (0)
pad2 . SetBottomMargin (0.25)

h1 . GetXaxis (). SetLabelSize (0)
h1 . GetXaxis (). SetTitleSize (0)
h1 . GetYaxis (). SetTitleSize (0.05)
h1 . GetYaxis (). SetLabelSize (0.03)
h1 . GetYaxis (). SetTitleOffset (0.6)


h3 . SetTitle ("")
h3 . GetXaxis (). SetLabelSize (0.12)
h3 . GetXaxis (). SetTitleSize (0.12)
h3 . GetYaxis (). SetLabelSize (0.1)
h3 . GetYaxis (). SetTitleSize (0.15)
#ratio . GetYaxis (). SetTitle (" Data /MC")
h3 . GetYaxis (). SetTitleOffset (0.17)
#207,512 divisions
h3 . GetYaxis (). SetNdivisions (207)
h3 . GetXaxis (). SetNdivisions (207)



canvas.Update()
canvas.Print("/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/picture.png")

## Lorenztian

In [ ]:
%jsroot on
from ROOT import TMath
data = low_pt_below_mid_rapidity_cut['mass']
ROOT.gStyle.SetOptStat(0);
h1 = ROOT.TH1F("B_&_S","",50,1.10,1.14)
h2 = ROOT.TH1F("h2", "h2", 50, 1.10, 1.14);
h3 = ROOT.TH1F("h2", "h2", 50, 1.10, 1.14);
for i in range(0,data.shape[0]):
    h1.Fill(data.iloc[i])

hs =ROOT.THStack("hs","");


pi=TMath.Pi()
f2 = TF1("full","((0.5)*[0]*[1]) /((x-[2])*(x-[2])+ .25*[1]*[1]) +[3]+[4]*x+[5]*x*x",1.08,1.2);
f2.SetNpx(1000);
f2.SetLineColor(TColor.GetColor(5))
f2.SetParameters(1,0.0001,1.1156,par1[1],par1[2],par1[3]);
c = ROOT.TCanvas("myCanvasName","The Canvas Title",1000,1000)
h1.Fit(f2,"MIRN");
#TVirtualFitter::SetDefaultFitter("Minuit");
#h1.SetFillColor(kGreen);
h1.SetFillStyle(3003);
h1.SetLineWidth(2)
Red = ROOT.TColor.GetColor(1)
#h1.SetFillColor(Red);
#h1.SetLineColor(Red)
h1.SetYTitle("Entries")
h1.SetLabelSize(0.04)
h2.SetMarkerColor(2)
#h1.SetTitle("Entries with Gaussian plus 2nd order polynom bac")
print("red chi2",f2.GetChisquare()/((bin2-bin1+1)-6))

par2 = f2.GetParameters()
hs.Add(h1);
c.Divide (1,2)
c.Draw()
c.cd(1);
h1.Draw("E")
fs = TF1("fs","((0.5)*[0]*[1]) /((x-[2])*(x-[2])+ .25*[1]*[1])",1.08,1.2);
fs.SetNpx(1000);
fb = TF1("fb","[0]+[1]*x+[2]*x*x",1.08,1.2)
fb.SetLineStyle(2)
fb.SetNpx(1000);
 
fs.SetParameters(f2.GetParameter(0),f2.GetParameter(1),f2.GetParameter(2));
fb.SetParameters(f2.GetParameter(3),f2.GetParameter(4),f2.GetParameter(5));
fs.Draw("SAME")
fb.Draw("SAME")
f2.Draw("SAME")
#h1.Sumw2();


bin1 = h1.FindBin(1.11);
bin2 = h1.FindBin(1.124);
for i in range(bin1,bin2):
    f_value= f2.Eval(h1.GetBinCenter(i));
    t_value = h1.GetBinContent(i)
    h2.SetBinContent(i,f_value)
    h3.SetBinContent(i,(t_value-f_value)/h1.GetBinError(i))

#h1.Draw("SAME")
#h1.SetLineColor(kRed);
h2.Sumw2()

hs.Add(h2);
#h2.Draw("SAME")
#hs.Draw("nostack")
#h2.Draw("SAME")
legend = ROOT.TLegend(0.9,0.9,0.6,0.5);
legend.AddEntry(h1,"Invariant mass of lambda","l");
legend.AddEntry(f2,"A #frac{0.5 #Gamma}{(m-m_{0})^{2} + 0.25#Gamma^{2}}+b+cx+dx^{2}","l");
legend.AddEntry(fs,"Ae^{-#frac{1}{2} #frac{(x-#mu)^{2}}{#sigma^{2}}}","l");
legend.AddEntry(fb,"b+cx+dx^{2}","l");
legend.Draw()
#rp = ROOT.TRatioPlot(h1, h2);
#rp.Draw("E1");
#h1.Sumw2();
c.cd(2)
#h3=(h1-h2)
h3.SetLineColor(TColor.GetColor(5))
h3.SetYTitle("D-f/dl")
h3.Draw()
f0 = TF1("f0","[0]",1.08,1.2)
f0.SetParameters(0)
f0.Draw("SAME")


c.Update()

In [ ]:
def pT_vs_rapidity(df, var_xaxis , var_yaxis , range_var_xaxis, range_var_yaxis):
    import matplotlib as mpl
    fig, axs = plt.subplots(figsize=(15, 10))
    h=plt.hist2d(df[var_xaxis],df[var_yaxis],range=[range_var_xaxis,range_var_yaxis], bins=100, norm=mpl.colors.LogNorm())
    cbar = fig.colorbar(h[3])
    plt.vlines(x=1.59,ymin=-1,ymax=4, color='r', linestyle='-')
    #plt.vlines(x=2.0,ymin=-1,ymax=4, color='r', linestyle='-')
    plt.hlines(y=0.4, xmin=-0.1, xmax=3.5, colors='b', linestyles='solid', label='')
    #plt.hlines(y=0.9, xmin=-0.1, xmax=3.5, colors='b', linestyles='solid', label='')
    plt.xlabel(''+var_xaxis, fontsize=15)
    plt.ylabel(''+var_yaxis, fontsize=15)
    plt.show()
    fig.tight_layout()
    fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity.png")

In [ ]:
range1=[-0.1, 3.5]
range2=[-0.5, 3.5]

pT_vs_rapidity(df3_base,'rapidity','pT', range1, range2)

In [ ]:
pt_rapidity_cut = df3_base[(df3_base['rapidity']<2) & (df3_base['rapidity']>0.8) &(df3_base['pT']>0.15)
                           &(df3_base['pT']<0.9)]
pT_vs_rapidity(pt_rapidity_cut,'rapidity','pT', range1, range2)

In [ ]:
high_pt_above_mid_rapidity_cut = df3_base[(df3_base['rapidity']>1.5996) & (df3_base['pT']>0.9)]
pT_vs_rapidity(high_pt_above_mid_rapidity_cut,'rapidity','pT', range1, range2)

In [ ]:

pT_vs_rapidity(low_pt_below_mid_rapidity_cut,'rapidity','pT', range1, range2)